In [1]:
import numpy as np
import opendssdirect as dss
import matplotlib.pyplot as plt
from lib.NR3 import NR3_function as NR31
from lib.compute_simultaneously import NR3_function as NR32
from lib.network_mapper_OpenDSSnonvec import network_mapper_function as nmf2
from lib.network_mapper import network_mapper_function as nmf

from math import tan,acos
import copy
import pandas as pd
import time
import re
import sys

In [2]:
fp = "C:\\Users\\kathl\\Desktop\\LinDist3Flow\\20180601\\PYTHON\\"
#fp = '/home/michael/Desktop/git/LinDist3Flow/20180601/NETWORKS/'
#fn = 'compare_lindist3flow_05node_threephase_unbalanced_oscillation_03.txt'
fn = '02node_threephase_unbalanced.dss'
#fn = 'lindist3flow_ieee13.txt'
network1 = nmf(fp, fn) #using LinDist3Flow config

nnode = network1.nodes.nnode
print('nnode: ', nnode)
nline = network1.lines.nline
print('nline: ', nline)

network1.cons.wpu = np.zeros((3,nnode))
network1.vvc.vvcpu = np.zeros((3,nnode))


network1.loads.aPQ = 1.00*np.ones((3,nnode))*(network1.loads.spu != 0)
network1.loads.aI = 0.00*np.ones((3,nnode))*(network1.loads.spu != 0)
network1.loads.aZ = 0.00*np.ones((3,nnode))*(network1.loads.spu != 0)


# network1.loads.spu = 1.25*network1.loads.spu

slackidx = 0
Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])


VNR01, INR01, STXNR01, SRXNR01, iNR01, sNR01, itercount01 = \
    NR31(network1,slackidx,Vslack,None,None,None,None)


print('VNR01\n', np.round(VNR01,decimals=6))
print('INR01\n', np.round(INR01,decimals=6))
print('STXNR01\n', np.round(STXNR01,decimals=6))
print('SRXNR01\n', np.round(SRXNR01,decimals=6))
print('iNR01\n', np.round(iNR01,decimals=6))
print('sNR01\n', np.round(sNR01,decimals=6))
print('itercount01:', itercount01)

print('|VNR01|\n', np.round(np.abs(VNR01),decimals=6))
print('<VNR01\n', np.round(180/np.pi*np.angle(VNR01),decimals=6))
print('D<VNR01\n', 180/np.pi*np.angle(VNR01) - 180/np.pi*np.angle(VNR01[:,[0]]))

UnboundLocalError: local variable 'Sbase' referenced before assignment

In [ ]:
network1 = nmf2('IEEE_13Node_Modified_01.dss', -1)
#network1 = network_mapper_function('compare__lindist3flow_05node_threephase_unbalanced_oscillation_03.dss')

nnode = network1.nodes.nnode
nline = network1.lines.nline

nnode = network1.nodes.nnode
print('nnode: ', nnode)
nline = network1.lines.nline
print('nline: ', nline)

network1.cons.wpu = np.zeros((3,nnode))
network1.vvc.vvcpu = np.zeros((3,nnode))

# network1.loads.aPQ = .750*np.ones((3,nnode))*(network1.loads.spu != 0)
# network1.loads.aI = 0.10*np.ones((3,nnode))*(network1.loads.spu != 0)
# network1.loads.aZ = 0.15*np.ones((3,nnode))*(network1.loads.spu != 0)

network1.loads.aPQ = 1.00*np.ones((3,nnode))*(network1.loads.spu != 0)
network1.loads.aI = 0.00*np.ones((3,nnode))*(network1.loads.spu != 0)
network1.loads.aZ = 0.00*np.ones((3,nnode))*(network1.loads.spu != 0)

# network1.loads.spu = 1.25*network1.loads.spu

slackidx = 0
Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])

fn = 'IEEE_13Node_Modified_01.dss'
VNR02, iNR02, sNR02, STXNR02, SRXNR02, INR02,itercount02 = NR32(network1, fn, slackidx,Vslack,None,None,None,None) #using OpenDSS as network object


# VNR02, INR02, STXNR02, SRXNR02, iNR02, sNR02, itercount02 = \
#     NR31(network1,slackidx,Vslack,None,None,None,None)


print('VNR02\n', np.round(VNR02,decimals=6))
print('INR02\n', np.round(INR02,decimals=6))
print('STXNR02\n', np.round(STXNR02,decimals=6))
print('SRXNR02\n', np.round(SRXNR02,decimals=6))
print('iNR02\n', np.round(iNR02,decimals=6))
print('sNR02\n', np.round(sNR02,decimals=6))
print('itercount02:', itercount02)

print('|VNR02|\n', np.round(np.abs(VNR02),decimals=6))
print('<VNR02\n', np.round(180/np.pi*np.angle(VNR02),decimals=6))
print('D<VNR02\n', 180/np.pi*np.angle(VNR02) - 180/np.pi*np.angle(VNR02[:,[0]]))

In [ ]:
Vbase = 2401.7771198288433
Sbase = 1000000.0
Ibase = Sbase/Vbase
Zbase = Vbase/Ibase

print(Ibase)
print(Zbase)

dss.run_command('Redirect IEEE_13Node_Modified_01.dss') #Using OpenDSS

# Set slack bus (sourcebus) voltage reference in p.u.
SlackBusVoltage = 1.000
dss.Vsources.PU(SlackBusVoltage)

dss.Solution.Convergence(0.000000000001)

# Solve power flow with OpenDSS file
dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')
else:
    print('Initial Model Converged. Proceeding to Next Step.')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    dss.Monitors.ResetAll()
    
    #set solution Params
    #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
    dss.Solution.Mode(1)
    dss.Solution.Number(1)
    dss.Solution.StepSize(1)
    dss.Solution.ControlMode(-1)
    dss.Solution.MaxControlIterations(1000000)
    dss.Solution.MaxIterations(30000)
    #Easy process to get all names and count of loads, a trick to avoid
    #some more lines of code
    TotalLoads=dss.Loads.Count()
    AllLoadNames=dss.Loads.AllNames()
    print('OpenDSS Model Compliation Done.')
    print('Iterations: ', dss.Solution.Iterations())
    print('Tolerance: ', dss.Solution.Convergence())

print('')

# print(dss.Solution.Converged())

# Print number of buses, and bus names
print(len(dss.Circuit.AllBusNames()))
print(dss.Circuit.AllBusNames())

# Print number of loads, and load names
print(len(dss.Loads.AllNames()))
print(dss.Loads.AllNames())

print('')

VDSS = np.zeros((3,len(dss.Circuit.AllBusNames())),dtype='complex')

for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
#     print(dss.Circuit.AllBusNames()[k1])
#     print(dss.Bus.Nodes())

#     print('puVOTLAGES - LN CARTESIAN')
#     print(dss.Bus.PuVoltage())

    ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1

    Vtemp = np.asarray(dss.Bus.PuVoltage())
    Vtemp = Vtemp[0:5:2] + 1j*Vtemp[1:6:2]
    
#     print(np.asarray(dss.Bus.Nodes(),'int'))
    
    VDSS[ph,k1] = Vtemp    
        
#     VDSS[np.asarray(dss.Bus.Nodes(),'int'),k1] = np.array(dss.Bus.PuVoltage()[0:5:2] + 1j*dss.Bus.PuVoltage()[1:6:2])
    
    
#     VDSS[dss.Bus.Nodes()-1,k1] = dss.Bus.PuVoltage()[0:2:5]
#     for k2 in range(len(dss.Bus.Nodes())):
#         VDSS[int(dss.Bus.Nodes()[k2])-1,k1] = dss.Bus.PuVoltage()[2*k2] + 1j*dss.Bus.PuVoltage()[2*k2+1]
        
print('VDSS\n', np.round(VDSS,decimals=6))


IDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

for k1 in range(len(dss.Lines.AllNames())):
    dss.Lines.Name(dss.Lines.AllNames()[k1])
#     print(dss.Lines.AllNames()[k1])
    ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
    Imn = np.asarray(dss.CktElement.Currents())/Ibase
#     print(Imn)
    Imn = Imn[0:int(len(Imn)/2)]
#     print(Imn)
    Imn = Imn[0:5:2] + 1j*Imn[1:6:2]
#     print(Imn)
    IDSS[ph,k1] = Imn
#     print('')

print('IDSS\n', np.round(IDSS,decimals=6))

STXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')
SRXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

for k1 in range(len(dss.Lines.AllNames())):
    dss.Lines.Name(dss.Lines.AllNames()[k1])
#     print(dss.Lines.AllNames()[k1])
#     print(dss.CktElement.BusNames())
    ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
#     print(ph)
    Sk = np.asarray(dss.CktElement.Powers())/(Sbase/1000)
#     print(Sk)
    
#     print(Sk[0:int(len(Sk)/2)])
#     print(Sk[int(len(Sk)/2):])
    
    STXtemp = Sk[0:int(len(Sk)/2)]
    SRXtemp = Sk[int(len(Sk)/2):]
    
    STXtemp = STXtemp[0:5:2] + 1j*STXtemp[1:6:2]
#     print(STXtemp)
    
    SRXtemp = -(SRXtemp[0:5:2] + 1j*SRXtemp[1:6:2])
#     print(SRXtemp)
    
    STXDSS[ph,k1] = STXtemp
    SRXDSS[ph,k1] = SRXtemp

print('STXDSS\n', np.round(STXDSS,decimals=6))
print('SRXDSS\n', np.round(SRXDSS,decimals=6))

print('|VDSS|\n', np.round(np.abs(VDSS),decimals=6))
print('<VDSS\n', np.round(180/np.pi*np.angle(VDSS),decimals=6))
print('D<VDSS\n', 180/np.pi*np.angle(VDSS) - 180/np.pi*np.angle(VDSS[:,[0]]))

In [ ]:
#dss - opendss solution
#01 - nonvectorized solution using LinDist3Flow file network mapper
#02 - nonvectorized solution using OpenDSS file network mapper
print('Complex Voltage difference')
print(np.max(np.abs(VNR01 - VNR02)))
print(np.max(np.abs(VDSS - VNR01))) 
print(np.max(np.abs(VDSS - VNR02)))

print('Complex Current difference')
print(np.max(np.abs(INR01 - INR02)))
print(np.max(np.abs(IDSS - INR01)))
print(np.max(np.abs(IDSS - INR02)))

print('Complex TX Power difference')
print(np.max(np.abs(STXNR01 - STXNR02)))
print(np.max(np.abs(STXDSS - STXNR01)))
print(np.max(np.abs(STXDSS - STXNR02)))

print('Complex RX Power difference')
print(np.max(np.abs(SRXNR01 - SRXNR02)))
print(np.max(np.abs(SRXDSS - SRXNR01)))
print(np.max(np.abs(SRXDSS - SRXNR02)))

print('Voltage Magnitude difference')
print(np.max(np.abs(np.abs(VNR01) - np.abs(VNR02))))
print(np.max(np.abs(np.abs(VDSS) - np.abs(VNR01))))
print(np.max(np.abs(np.abs(VDSS) - np.abs(VNR02))))

print('Voltage Angle difference')
print(np.max(np.abs(180/np.pi*np.angle(VNR01) - 180/np.pi*np.angle(VNR02))))
print(np.max(np.abs(180/np.pi*np.angle(VDSS) - 180/np.pi*np.angle(VNR01))))
print(np.max(np.abs(180/np.pi*np.angle(VDSS) - 180/np.pi*np.angle(VNR02))))

In [ ]:
print(VNR01.shape)
print(VNR02.shape)
print(VDSS.shape)